# Imports

In [1]:
import fiftyone as fo
import fiftyone.utils.coco as fouc
from fiftyone.core.labels import Detection
from PIL import Image

import torch
import torch.nn as nn
import torch.utils
import torch.utils.data

import torchvision
from torchvision.models.segmentation import fcn_resnet50, FCN_ResNet50_Weights
from torchvision.models.detection import fasterrcnn_resnet50_fpn, FasterRCNN_ResNet50_FPN_Weights
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import matplotlib.pyplot as plt

import os
import sys
import importlib
import time
from pathlib import Path
from datetime import datetime

import utils
import train
from engine import train_one_epoch, evaluate
#from train import train_one_epoch, evaluate
import transforms
sys.path.insert(0, '..')
import DatasetLoaders as DatasetLoaders
importlib.reload(DatasetLoaders)
from DatasetLoaders import HRSIDDetectionDataset
importlib.reload(utils)
importlib.reload(transforms)
importlib.reload(train)

<module 'train' from '/home/k3vinli/ENEE439/Capstone/models/faster_rcnn_resnet/train.py'>

# Loading Data

In [2]:
p = Path()
top_dir = p.absolute().parents[1]
top_dir

PosixPath('/home/k3vinli/ENEE439/Capstone')

In [3]:
# Loading Dataset
name = "HRSID"
if name in fo.list_datasets():
    dataset_traintest = fo.load_dataset(name)
else:
    dataset_dir = top_dir / "Datasets" / "HRSID"
    # The type of the dataset being imported
    dataset_type = fo.types.COCODetectionDataset

    dataset_traintest = fo.Dataset.from_dir(
        dataset_dir=dataset_dir,
        dataset_type=dataset_type,
        name=name,
    )

In [4]:
# dataset_traintest.delete()

In [4]:
name = "HRSID_train"
if name in fo.list_datasets():
    dataset_train = fo.load_dataset(name)
else:
    dataset_dir = top_dir / "Datasets" / "HRSID"
    label_path = top_dir / "Datasets"/ "HRSID" / "annotations" / "train2017.json"
    # The type of the dataset being imported
    dataset_type = fo.types.COCODetectionDataset

    dataset_train = fo.Dataset.from_dir(
        dataset_dir=dataset_dir,
        dataset_type=dataset_type,
        name=name,
        labels_path=label_path
    )

In [5]:
name = "HRSID_test"
if name in fo.list_datasets():
    dataset_test = fo.load_dataset(name)
else:
    dataset_dir = top_dir / "Datasets" / "HRSID"
    label_path = top_dir / "Datasets"/ "HRSID" / "annotations" / "test2017.json"
    # The type of the dataset being imported
    dataset_type = fo.types.COCODetectionDataset

    dataset_test = fo.Dataset.from_dir(
        dataset_dir=dataset_dir,
        dataset_type=dataset_type,
        name=name,
        labels_path=label_path
    )

In [20]:
# dataset_traintest.delete()

In [14]:
dataset_train.compute_metadata()
dataset_test.compute_metadata()

In [9]:
session = fo.launch_app()

In [6]:
def get_model(num_classes):
    weights = FasterRCNN_ResNet50_FPN_Weights.DEFAULT
    transforms = weights.transforms()
    model = fasterrcnn_resnet50_fpn(weights=weights)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

In [10]:
def do_training(model, torch_dataset, torch_dataset_test, num_epochs=4):
    # define training and validation data loaders
    data_loader = torch.utils.data.DataLoader(
        torch_dataset, batch_size=10, shuffle=True, num_workers=2,
        collate_fn=utils.collate_fn)
    
    data_loader_test = torch.utils.data.DataLoader(
        torch_dataset_test, batch_size=10, shuffle=False, num_workers=2,
        collate_fn=utils.collate_fn)

    # train on the GPU or on the CPU, if a GPU is not available
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    print("Using device %s" % device)

    # move model to the right device
    model.to(device)

    # construct an optimizer
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.005,
                                momentum=0.9, weight_decay=0.0005)
    # and a learning rate scheduler
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                    step_size=3,
                                                    gamma=0.1)

    for epoch in range(num_epochs):
        # train for one epoch, printing every 10 iterations
        train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)

        # update the learning rate
        lr_scheduler.step()
        # evaluate on the test dataset
        evaluate(model, data_loader_test, device=device)

In [7]:
train_transforms = transforms.Compose([transforms.PILToTensor(), transforms.ConvertImageDtype(torch.float)])
test_transforms = transforms.Compose([transforms.PILToTensor(), transforms.ConvertImageDtype(torch.float)])

In [8]:
torch_dataset = HRSIDDetectionDataset(dataset_train, train_transforms)
torch_dataset_test = HRSIDDetectionDataset(dataset_test, test_transforms)

In [16]:
model = get_model(2)

In [13]:
do_training(model, torch_dataset, torch_dataset_test, num_epochs=15)

Using device cuda
Epoch: [0]  [  0/365]  eta: 0:32:40  lr: 0.000019  loss: 1.0524 (1.0524)  loss_classifier: 0.5767 (0.5767)  loss_box_reg: 0.1412 (0.1412)  loss_objectness: 0.2819 (0.2819)  loss_rpn_box_reg: 0.0526 (0.0526)  time: 5.3706  data: 0.5320  max mem: 6243
Epoch: [0]  [ 10/365]  eta: 0:15:47  lr: 0.000156  loss: 1.0524 (1.1729)  loss_classifier: 0.4788 (0.4476)  loss_box_reg: 0.1210 (0.1333)  loss_objectness: 0.2819 (0.5434)  loss_rpn_box_reg: 0.0514 (0.0486)  time: 2.6703  data: 0.0728  max mem: 6402
Epoch: [0]  [ 20/365]  eta: 0:12:58  lr: 0.000293  loss: 0.6852 (0.9050)  loss_classifier: 0.2497 (0.3290)  loss_box_reg: 0.1688 (0.1677)  loss_objectness: 0.1673 (0.3573)  loss_rpn_box_reg: 0.0387 (0.0512)  time: 2.1021  data: 0.0277  max mem: 6402
Epoch: [0]  [ 30/365]  eta: 0:11:43  lr: 0.000430  loss: 0.5103 (0.7692)  loss_classifier: 0.1663 (0.2748)  loss_box_reg: 0.1982 (0.1766)  loss_objectness: 0.0726 (0.2689)  loss_rpn_box_reg: 0.0370 (0.0489)  time: 1.7848  data: 0.02

In [14]:
data_loader_test = torch.utils.data.DataLoader(
                        torch_dataset_test, batch_size=1, shuffle=False, num_workers=2,
                        collate_fn=utils.collate_fn)
evaluate(model, data_loader_test, device=torch.device('cuda'))

creating index...
index created!
Test:  [   0/1961]  eta: 0:13:24  model_time: 0.3172 (0.3172)  evaluator_time: 0.0027 (0.0027)  time: 0.4104  data: 0.0873  max mem: 6404
Test:  [ 100/1961]  eta: 0:01:45  model_time: 0.0457 (0.0490)  evaluator_time: 0.0015 (0.0015)  time: 0.0530  data: 0.0038  max mem: 6404
Test:  [ 200/1961]  eta: 0:01:36  model_time: 0.0458 (0.0477)  evaluator_time: 0.0013 (0.0015)  time: 0.0525  data: 0.0038  max mem: 6404
Test:  [ 300/1961]  eta: 0:01:29  model_time: 0.0457 (0.0472)  evaluator_time: 0.0015 (0.0015)  time: 0.0529  data: 0.0037  max mem: 6404
Test:  [ 400/1961]  eta: 0:01:23  model_time: 0.0456 (0.0468)  evaluator_time: 0.0014 (0.0016)  time: 0.0529  data: 0.0038  max mem: 6404
Test:  [ 500/1961]  eta: 0:01:18  model_time: 0.0456 (0.0467)  evaluator_time: 0.0013 (0.0016)  time: 0.0531  data: 0.0038  max mem: 6404
Test:  [ 600/1961]  eta: 0:01:13  model_time: 0.0462 (0.0468)  evaluator_time: 0.0012 (0.0016)  time: 0.0586  data: 0.0046  max mem: 6404
T

In [13]:
def convert_torch_predictions(preds, det_id, s_id, w, h, classes, ignore_thin_bbox=False):
    # Convert the outputs of the torch model into a FiftyOne Detections object
    dets = []
    for bbox, label, score in zip(
        preds["boxes"].cpu().detach().numpy(), 
        preds["labels"].cpu().detach().numpy(), 
        preds["scores"].cpu().detach().numpy()
    ):
        # Parse prediction into FiftyOne Detection object
        x0,y0,x1,y1 = bbox
        if ignore_thin_bbox:
            if (x1-x0) < 1 or (y1-y0) < 1:
                continue
        coco_obj = fouc.COCOObject(det_id, s_id, int(label), [x0, y0, x1-x0, y1-y0])
        det = coco_obj.to_detection((w,h), classes)
        det["confidence"] = float(score)
        dets.append(det)
        det_id += 1
        
    detections = fo.Detections(detections=dets)
        
    return detections, det_id

In [12]:
def add_detections(model, torch_dataset, view, field_name="predictions", ignore_thin_bbox=False):
    # Run inference on a dataset and add results to FiftyOne
    torch.set_num_threads(1)
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    print("Using device %s" % device)

    model.eval()
    model.to(device)
    image_paths = torch_dataset.img_paths
    classes = torch_dataset.classes
    det_id = 0
    
    with fo.ProgressBar() as pb:
        for img, targets in pb(torch_dataset):
            # Get FiftyOne sample indexed by unique image filepath
            img_id = int(targets["image_id"][0])
            img_path = image_paths[img_id]
            sample = view[img_path]
            s_id = sample.id
            w = sample.metadata["width"]
            h = sample.metadata["height"]
            
            # Inference
            preds = model(img.unsqueeze(0).to(device))[0]
            
            detections, det_id = convert_torch_predictions(
                preds, 
                det_id, 
                s_id, 
                w, 
                h, 
                classes,
            )
            
            sample[field_name] = detections
            sample.save()

In [10]:
session.view = dataset_test.sort_by("eval_frcnn_fp", reverse=True)

In [47]:
file_name = f"{datetime.now().strftime('%m_%d_%y, %H-%M-%S')}e15.pt"
file_name


'04_21_23, 19-34-11e15.pt'

In [48]:
# https://pytorch.org/tutorials/beginner/saving_loading_models.html
# Saving Model
torch.save(model.state_dict(), os.path.abspath(f"faster_rcnn_resnet_weights/{file_name}"))

# LOADING MODEL

In [9]:
# loading the model from saved state
PATH = "faster_rcnn_resnet_weights/23_04_21--19-38-02e15.pt"
model = get_model(2)
model.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [17]:
add_detections(model, torch_dataset_test, dataset_test, field_name="predictions_frcnn")

Using device cuda
 100% |███████████████| 1961/1961 [3.0m elapsed, 0s remaining, 13.6 samples/s]      


In [42]:
dataset_test

Name:        HRSID_test
Media type:  image
Num samples: 1961
Persistent:  False
Tags:        []
Sample fields:
    id:                  fiftyone.core.fields.ObjectIdField
    filepath:            fiftyone.core.fields.StringField
    tags:                fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:            fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    detections:          fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    segmentations:       fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    predictions_lee:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    seg_predictions_lee: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    eval_seg2_tp:        fiftyone.core.fields.IntField
    eval_seg2_fp:        fiftyone.core.fields.IntField
    eval_seg2_fn:        fiftyone.core.fields.IntField
    ev

In [45]:
dataset_train

Name:        HRSID_train
Media type:  image
Num samples: 3642
Persistent:  False
Tags:        []
Sample fields:
    id:                  fiftyone.core.fields.ObjectIdField
    filepath:            fiftyone.core.fields.StringField
    tags:                fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:            fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    detections:          fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    segmentations:       fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    predictions_lee:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    seg_predictions_lee: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    eval_seg_tp:         fiftyone.core.fields.IntField
    eval_seg_fp:         fiftyone.core.fields.IntField
    eval_seg_fn:         fiftyone.core.fields.IntField
    e

In [22]:
results = fo.evaluate_detections(
    dataset_test, 
    "predictions_frcnn", 
    classes=["ship"],
    eval_key="eval_frcnn", 
    compute_mAP=True,
    gt_field='detections'
)

Evaluating detections...
 100% |███████████████| 1961/1961 [43.4s elapsed, 0s remaining, 73.2 samples/s]       
Performing IoU sweep...
 100% |███████████████| 1961/1961 [25.0s elapsed, 0s remaining, 101.8 samples/s]      


In [23]:
results.mAP()

0.6131548779835022

In [24]:
results.print_report()

              precision    recall  f1-score   support

        ship       0.20      0.91      0.33      5921

   micro avg       0.20      0.91      0.33      5921
   macro avg       0.20      0.91      0.33      5921
weighted avg       0.20      0.91      0.33      5921



In [25]:
add_detections(model, torch_dataset, dataset_train, field_name="predictions_frcnn")

Using device cuda
 100% |███████████████| 3642/3642 [5.7m elapsed, 0s remaining, 11.8 samples/s]      


In [26]:
results = fo.evaluate_detections(
    dataset_train, 
    "predictions_frcnn", 
    classes=["ship"],
    eval_key="eval_frcnn", 
    compute_mAP=True,
    gt_field='detections'
)

Evaluating detections...
 100% |███████████████| 3642/3642 [1.4m elapsed, 0s remaining, 71.7 samples/s]      
Performing IoU sweep...
 100% |███████████████| 3642/3642 [45.3s elapsed, 0s remaining, 92.6 samples/s]      


In [27]:
results.mAP()

0.6670056311532374

In [28]:
results.print_report()

              precision    recall  f1-score   support

        ship       0.21      0.93      0.34     11047

   micro avg       0.21      0.93      0.34     11047
   macro avg       0.21      0.93      0.34     11047
weighted avg       0.21      0.93      0.34     11047



# small set

In [10]:
small_set = dataset_test[:8]
torch_small_set = HRSIDDetectionDataset(small_set, transforms=test_transforms)

In [14]:
add_detections(model, torch_small_set, small_set, field_name="predictions")


Using device cuda
 100% |█████████████████████| 8/8 [2.4s elapsed, 0s remaining, 3.3 samples/s]      


# Ignoring bboxes on edges
evaluation with bboxes on edges ignored. Done by calculating if the bbox is just a line, since most falsely detected bboxes on the edges are lines

In [46]:
add_detections(model, torch_dataset_test, dataset_test, field_name="predictions_frcnn_bbox_ignore")

Using device cuda
 100% |███████████████| 1961/1961 [3.3m elapsed, 0s remaining, 9.2 samples/s]      


In [47]:
results = fo.evaluate_detections(
    dataset_test, 
    "predictions_frcnn_bbox_ignore", 
    classes=["ship"],
    eval_key="eval_frcnn_bbox_ignore", 
    compute_mAP=True,
    gt_field='detections'
)

Sample field 'eval_frcnn_bbox_ignore_tp' does not exist
Sample field 'eval_frcnn_bbox_ignore_fp' does not exist
Sample field 'eval_frcnn_bbox_ignore_fn' does not exist
Evaluating detections...
 100% |███████████████| 1961/1961 [44.2s elapsed, 0s remaining, 66.3 samples/s]       
Performing IoU sweep...
 100% |███████████████| 1961/1961 [25.8s elapsed, 0s remaining, 98.0 samples/s]       


In [48]:
print(results.mAP())
results.print_report()

0.6131548779835022
              precision    recall  f1-score   support

        ship       0.20      0.91      0.33      5921

   micro avg       0.20      0.91      0.33      5921
   macro avg       0.20      0.91      0.33      5921
weighted avg       0.20      0.91      0.33      5921



In [49]:
add_detections(model, torch_dataset, dataset_train, field_name="predictions_frcnn_bbox_ignore")

Using device cuda
 100% |███████████████| 3642/3642 [6.1m elapsed, 0s remaining, 9.8 samples/s]       


In [50]:
results = fo.evaluate_detections(
    dataset_train, 
    "predictions_frcnn_bbox_ignore", 
    classes=["ship"],
    eval_key="eval_frcnn_bbox_ignore", 
    compute_mAP=True,
    gt_field='detections'
)

Evaluating detections...
 100% |███████████████| 3642/3642 [1.4m elapsed, 0s remaining, 70.1 samples/s]      
Performing IoU sweep...
 100% |███████████████| 3642/3642 [45.6s elapsed, 0s remaining, 91.0 samples/s]      


In [51]:
print(results.mAP())
results.print_report()

0.6670056311532374
              precision    recall  f1-score   support

        ship       0.21      0.93      0.34     11047

   micro avg       0.21      0.93      0.34     11047
   macro avg       0.21      0.93      0.34     11047
weighted avg       0.21      0.93      0.34     11047

